# "Creating Bounding Boxes from Class Activation Maps "
> "Step by step tutorial on how to create bounding boxes from Class Activation Maps"
- toc: true
- branch: master
- badges: true
- comments: true
- categories: [cmaps, medical_imaging, dicoms, bounding_boxes]
- image: images/bb.PNG

This notebook uses the approach of generating bounding boxes from Class Activation Maps (CAM). CAM was introduced by Bolei Zhou et al. in the paper [Learning Deep Features for Discriminative Localization](https://arxiv.org/abs/1512.04150) and is a great tool for model interpretation.

Grad CAM is a variation of CAM that was introduced in [Grad-CAM: Why Did You Say That? Visual Explanations from Deep Networks via Gradient-based Localization](https://arxiv.org/abs/1611.07450)

Grad CAM is similar to CAM in that it is a weighted combination of feature maps but followed by a ReLU. The output of Grad CAM is a “class-discriminative localization map” where the hot part of the heatmap corresponds to a particular class.

In [3]:
#hide
from fastai.vision.all import *
from fastai.medical.imaging import *
from fmi.preprocessing import *
from fmi.train import *
from fmi.examine import *
import pydicom

matplotlib.rcParams['image.cmap'] = 'viridis'

CAM uses the output of the last convolutional layer to provide a heatmap visualization. A way of acccessing the activations during the training phase in Pytorch is done by using a hook. A hook is basically a function that is executed when either the forward or backward pass is called.

`Fastai` conveniently has a `Hook` class

In [4]:
class Hook():
    def __init__(self, m):
        self.hook = m.register_forward_hook(self.hook_func)   
    def hook_func(self, m, i, o): self.stored = o.detach().clone()
    def __enter__(self, *args): return self
    def __exit__(self, *args): self.hook.remove()

The gradients of every layer are calculated by PyTorch during the backward pass. To access the gradients you can register a hook on the backward pass and store these gradients. 

Again conveniently `fastai` has the `HookBwd` class.

In [5]:
class HookBwd():
    def __init__(self, m):
        self.hook = m.register_backward_hook(self.hook_func)   
    def hook_func(self, m, gi, go): self.stored = go[0].detach().clone()
    def __enter__(self, *args): return self
    def __exit__(self, *args): self.hook.remove()